In [15]:
import cv2
from scipy.ndimage.filters import convolve 
import numpy as np

In [30]:
def background_subtraction(frame):
    fgbg1 = cv2.bgsegm.createBackgroundSubtractorMOG();
    bframe=fgbg1.apply(frame)
    return bframe
    
frames=[]
cap=cv2.VideoCapture('running.avi')
while True:
    ret,frame=cap.read()
    
    #grayFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    frames.append(frame)
    if not ret:
        break
    cv2.destroyAllWindows() 
    cap.release()
bframes=[]
for frame in frames:
    bframe=background_subtraction(frame)
    


In [32]:
def gaussian_blurOperator(sigma): 
  size = 2*np.ceil(3*sigma)+1 
  x, y = np.mgrid[-size//2 + 1:size//2 + 1, -size//2 + 1:size//2 + 1] 
  g = np.exp(-((x**2 + y**2)/(2.0*sigma**2))) / (2*np.pi*sigma**2)
  return g/g.sum()

def generate_octave(init_level, s, sigma): 
  octave = [init_level] 
  k = 2**(1/s) 
  kernel = gaussian_blurOperator(k * sigma) 
  for _ in range(s+2): 
    next_level = convolve(octave[-1], kernel) 
    octave.append(next_level) 
  return octave
def generate_gaussian_pyramid(im, num_octave, s, sigma): 
  pyr = [] 
  for _ in range(num_octave): 
    octave = generate_octave(im, s, sigma) 
    pyr.append(octave) 
    im = octave[-3][::2, ::2] 
  return pyr

In [33]:
pyramidframes=[]
for image in bframes:
    pyramidframes=generate_gaussian_pyramid(image,4, s=3, sigma=1.6)
    